# Raw Inspection

Inspecting and screening the raw data, making annotations about:

- bad segments across time
- bad channels across the overall data

We will also automatically set the block breaks, and the recorded time
before the start and after the end of the experiment to "BAD".

Note 2020-07-11: could have used [mne.preprocessing.mark_flat](https://github.com/mne-tools/mne-python/blob/76fc838e954e70697f33cab0edbca61872ca1c23/mne/preprocessing/flat.py#L14-L104https://github.com/mne-tools/mne-python/blob/76fc838e954e70697f33cab0edbca61872ca1c23/mne/preprocessing/flat.py#L14-L104) to enhance visual inspection with some automatic "pre"-marking.

**THIS CODE IS MEANT TO BE RUN INTERACTIVELY**

**VISUAL INSPECTION BY AN EXPERT**


In [ ]:
%matplotlib qt

In [ ]:
import itertools
import os
import os.path as op

import mne
import numpy as np

from utils import BIDS_ROOT, task_not_present_for_subject

In [ ]:
# IO: Where to find the data
eeg_path_template = op.join(
    BIDS_ROOT, "sub-{0:02}", "eeg", "sub-{0:02}_task-{1}_eeg.vhdr"
)

# Where to save the annotations and bad channels
fname_annots_template = op.join(
    BIDS_ROOT, "derivatives", "sub-{0:02}", "sub-{0:02}_task-{1}_annotations.txt"
)
fname_channels_template = op.join(
    BIDS_ROOT, "derivatives", "sub-{0:02}", "sub-{0:02}_task-{1}_badchannels.txt"
)

In [ ]:
# Make a list of all data files to screen
subjects = range(1, 41)
tasks = ["ActiveFixed", "ActiveVariable", "YokedFixed", "YokedVariable", "description"]

combinations = list(itertools.product(subjects, tasks))

for combo in combinations:
    if task_not_present_for_subject(*combo):
        combinations.remove(combo)

print("{} datsets to screen overall".format(len(combinations)))

In [ ]:
# increment this index whenever one subject/task combination has been screened
current_work_index = 1

In [ ]:
# Load the data for the current work index
subject, task = combinations[current_work_index]
fname_annots = fname_annots_template.format(subject, task)
fname_channels = fname_channels_template.format(subject, task)
if op.exists(fname_annots) or op.exists(fname_channels):
    print('Data exists. You may want to increment "current_work_index".\n\n\n')

fpath = eeg_path_template.format(subject, task)
raw = mne.io.read_raw_brainvision(fpath, preload=True)

# Suppress an automatic "average reference"
raw.set_eeg_reference(ref_channels=[])

# Set the EOG and ECG channels to their type
raw.set_channel_types({"ECG": "ecg", "HEOG": "eog", "VEOG": "eog"})

# Set a standard montage for plotting later
# NOTE: Could potentially set the "true" (i.e., measured, digitized) electrode
# coordinates from the experiment here ... but template positions seem fine
montage = mne.channels.make_standard_montage("standard_1020")

raw.set_montage(montage)

In [ ]:
# Get segments for 1) block breaks and 2) pre/post experiment recording
general_description = "BAD_break"
buffer_plus_minus = 1.0  # buffer in seconds around breaks that won't be marked
orig_time = raw.annotations.orig_time


def get_stim_onset(raw, stim, nth_stim=0):
    """Help to find onset of a stimulus in the data."""
    idx = (raw.annotations.description == stim).nonzero()[0][nth_stim]
    return idx, raw.annotations.onset[idx]


# Get data for pre experiment
_, start_onset = get_stim_onset(raw, "Stimulus/S  1")
recording_onset = raw.first_samp / raw.info["sfreq"]
start_duration = (start_onset - recording_onset) - buffer_plus_minus
if start_duration < 0:
    start_duration = start_onset - recording_onset

raw.annotations.append(recording_onset, start_duration, general_description)

# Get data for block breaks ... and post experiment
n_blocks = 5
for block in range(n_blocks):
    break_idx, break_onset = get_stim_onset(raw, "Stimulus/S 23", nth_stim=block)
    # next event after a break marks the end of the break
    if block < n_blocks - 1:
        break_offset = raw.annotations.onset[break_idx + 1]
        break_duration = (break_offset - break_onset) - buffer_plus_minus
        if break_duration < 0:
            break_duration = break_offset - break_onset

    # unless for the last block, where we can extend the bad segment until
    # the end of the recording
    elif block == n_blocks - 1:
        break_offset = raw.last_samp / raw.info["sfreq"]
        break_duration = break_offset - break_onset

    if break_duration > buffer_plus_minus:
        break_onset += buffer_plus_minus

    raw.annotations.append(break_onset, break_duration, general_description)

In [ ]:
# Delete all other annotations except the BAD ones
# ... for focusing on data quality during visual inspection
# irrespective on current condition in the experiment
bad_idxs = [descr.startswith("BAD") for descr in raw.annotations.description]
to_delete = np.arange(len(raw.annotations))[~np.array(bad_idxs)]
raw.annotations.delete(to_delete)

In [ ]:
# Downsample the data to speed up plotting
raw.resample(sfreq=250)

In [ ]:
# Inspect power spectral density of the data
raw.plot_psd(reject_by_annotation=True)

In [ ]:
# Plot the data, interactively annotating
fig = raw.plot(n_channels=len(raw.ch_names), bad_color=(1, 0, 0), duration=20.0)

In [ ]:
# Whether to overwrite or not
overwrite = False

# Save the marked annotations and bad channels
if not op.exists(fname_annots) or overwrite:
    os.makedirs(op.split(fname_annots)[0], exist_ok=True)
    raw.annotations.save(fname_annots)
else:
    print(
        '{} already exists. Maybe increment "current_work_index".'.format(fname_annots)
    )

if not op.exists(fname_channels) or overwrite:
    os.makedirs(op.split(fname_channels)[0], exist_ok=True)
    with open(fname_channels, "w") as fout:
        lines = "\n".join(raw.info["bads"])
        fout.writelines(lines)
else:
    print(
        '{} already exists. Maybe increment "current_work_index".'.format(
            fname_channels
        )
    )